<a href="https://colab.research.google.com/github/matzmtok/Learning_XAI/blob/main/LIME%E3%82%92%E4%BD%BF%E3%81%A3%E3%81%A6%E6%A4%9C%E8%A8%BC%E3%81%99%E3%82%8B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 検証の目的

テーブルデータに対する判断タスクを考え、AIモデルが判別の根拠とした変数をLIMEで出力できるようになることを目指す。
* LIMEの動かし方
* LIMEの出力の味方
* LIMEの利点と弱点をふまえた使い所

# ライブラリの準備

判別のモデルの構築にはlightgbmを使用する
パッケージとして、pandas、numpy、matplotlib、seaborn、scikit-learnを使用する。

## まず pip でインストール

In [ ]:
!pip install lightgbm lime matplotlib seaborn scikit-learn pandas numpy

# 検証対象のデータ

Kaggleのタイタニック号生存者予測問題のデータセットを使用する

データの概要

|カラム名|カラムの意味|取扱いに関する備考|
|-----------|----------------------|---------------------|
|PassengerId|乗客ID                | AIモデルに入力しない|
|Suvived    |生存フラグ            |正解ラベル           |
|Pclass     |チケットクラス        |順序性がある変数として扱う|
|Name       |乗客の名前            |敬称を抽出・グルーピングして新たな変数Title|
|Sex        |性別                  |female, maleを0/1の変数に変換|
|Age        |年齢                  |一部欠損データがあるため中央値で補完|
|SibSp      |同乗者数（兄弟配偶者）|Parchと合算した新変数Familyに統合|
|Parch      |同乗者数（親・子）    |SibSpと合算した新変数Familyに統合|
|Ticket     |チケット番号          |AIモデルに入力しない|
|Fare       |運賃                  |一部欠損データあり|
|Cabin      |客室番号              |欠損多数のためAIモデルに入力しない|
|Embarked   |出港地                |欠損値あり、欠損データは使用しない。カテゴリ変数に変換|



In [ ]:
test = "/content/drive/MyDrive/Colab Notebooks/data/titanic/test.csv"
train = "/content/drive/MyDrive/Colab Notebooks/data/titanic/train.csv"

In [ ]:
import pandas as pd

df_test = pd.read_csv(test)
df_train = pd.read_csv(train)



# モデルの学習

まずモデルに学習させるためのデータの加工を行う
学習させるモデルにはLightGBMを用いる

## 前処理
LightGBMにはカテゴリカル変数を使用することが可能。
LIMEのライブラリの都合上、欠損値を入力できない


In [ ]:
from sklearn.preprocessing import OrdinalEncoder

def replace_name(series):
  """
   
  """
  series = series.apply(lambda x: x.split(',')[1].split('.')[0].strip())
  series = series.replace(['Capt', 'Col', 'Major', 'Dr', 'Rev'], 'Officer')
  series = series.replace(['Don', 'Sir', 'the Countess', 'Lady', 'Dona'], 'Royalty')

  return series

